# 初級②：属性 × コメントカテゴリ傾向分析

このセクションでは、GOLD テーブルを用いて、コメントカテゴリ（特にネガティブ／ポジティブカテゴリ）が属性（年代・性別など）とどう対応しているかを探索します。  
具体的には以下を目指します：

- 属性別ネガティブカテゴリ傾向を把握  
- ネガティブカテゴリーの構成を可視化  
- ポジティブカテゴリ別に満足度差を確認  
- 属性別ポジ／ネガ比率を比較  

In [0]:
CATALOG = "hiroshi"
SCHEMA  = "survey_analysis_simple"

# UCオブジェクト作成
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE {SCHEMA}")

GOLD_TABLE_NAME = "gold_survey_responses_final"

In [0]:
gold = spark.table(GOLD_TABLE_NAME)  # ← 実際の GOLD テーブル名
display(gold.limit(10))
gold.printSchema()

## 属性 × ネガティブカテゴリ 件数クロス集計

このセルでは、`age_group`（年代グループ）ごとに、各 `negative_feedback_category` の出現件数を集計します。  
次のセルで「割合」に変換して比較できるような布石です。

In [0]:
from pyspark.sql import functions as F

cross_neg = (
    gold
    .groupBy("age_group", "negative_feedback_category")
    .agg(F.count("*").alias("cnt"))
)

display(cross_neg)

## 年代ごとのネガティブカテゴリ割合算出と表示

このセルでは、年代別のネガティブ件数総数を取得し、クロス集計との結合を行って割合 `ratio` を算出します。  
年代間で不満カテゴリの比率の違いを見るためのデータを作ります。

In [0]:
# 年代ごとの総件数
total_by_age = (
    gold
    .groupBy("age_group")
    .agg(F.count("*").alias("total_cnt"))
)

# 割合を出す
cross_neg_ratio = (
    cross_neg
    .join(total_by_age, on="age_group")
    .withColumn("ratio", F.col("cnt") / F.col("total_cnt"))
    .orderBy("age_group", F.desc("ratio"))
)

display(cross_neg_ratio)

## 属性 × ネガティブカテゴリ割合を可視化（積み上げ棒グラフなど）

このセルでは、先ほどの割合データ `cross_neg_ratio` を用いて、属性（年代など）ごとにネガティブカテゴリ構成を視覚化します。  
ノートブック UI で “Visualization → Stacked Bar Chart” を選ぶと、年代ごとのカテゴリ構成が比較できます。

In [0]:
display(cross_neg_ratio)

## ポジティブカテゴリ別満足度比較（件数と平均値）

このセルでは、`positive_point_category` ごとに、満足度 (`satisfaction_int`) の平均と件数を算出し、  
どのポジティブカテゴリを挙げた人ほど満足度が高いかを見ます。

In [0]:
pos_satis = (
    gold
    .groupBy("positive_point_category")
    .agg(
        F.avg("satisfaction_int").alias("avg_satisfaction"),
        F.count("*").alias("cnt")
    )
    .orderBy(F.desc("cnt"))
)

display(pos_satis)

## ポジティブカテゴリ別満足度を可視化（棒グラフ）

このセルでは、`pos_satis` の出力を用いて、  
ノートブック UI 上で “Visualization → Bar chart” を選択することで、カテゴリ別満足度を視覚化できます。

In [0]:
display(pos_satis)

## 属性別ポジ／ネガ比率分析

このセルでは、性別（例）を属性に使い、  
その属性層でポジティブコメントを持つかどうか、ネガティブを持つかどうかの比率を算出します。  
比率比較によって、属性間の感想傾向差を見ます。

In [0]:
pos_neg_by_gender = (
    gold
    .groupBy("gender")
    .agg(
        F.count("*").alias("total_cnt"),
        F.sum(F.when(F.col("positive_point").isNotNull() & (F.col("positive_point") != ""), 1).otherwise(0)).alias("pos_count"),
        F.sum(F.when(F.col("negative_feedback").isNotNull() & (F.col("negative_feedback") != ""), 1).otherwise(0)).alias("neg_count")
    )
    .withColumn("pos_ratio", F.col("pos_count") / F.col("total_cnt"))
    .withColumn("neg_ratio", F.col("neg_count") / F.col("total_cnt"))
    .orderBy("gender")
)

display(pos_neg_by_gender)

## ✅ この章のまとめと次への道筋

**この章で得られる知見例**  
- 年代別に、どのネガティブカテゴリが目立つか  
- ポジティブカテゴリ別で満足度差はあるか  
- 属性（性別など）ごとのポジ／ネガ出現比率の違い  

**次に進むべき分析案**  
- 属性＋ネガティブカテゴリを特徴量にして購入意向予測モデルをつくる  
- LLM によるネガティブ文句の「本音・改善案要約」抽出  
- 複数カテゴリ混在コメントの重みづけ・スコア化  

この章でのクロス分析が、次フェーズ（モデル構築や LLM 分析）の足掛かりになるよ。